In [1]:

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


In [2]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [3]:


#discriminator input from real data
X = tf.placeholder(tf.float32, shape=[None, 784])

#discriminator parameters
D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

#Inner discriminator parameters
Id_W1 = tf.Variable(xavier_init([784, 128]))
Id_b1 = tf.Variable(tf.zeros(shape=[128]))

Id_W2 = tf.Variable(xavier_init([128, 1]))
Id_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_Id = [Id_W1, Id_W2, Id_b1, Id_b2]


#noise vector (generator input)
Z = tf.placeholder(tf.float32, shape=[None, 100])

#generator parameters
G_W1 = tf.Variable(xavier_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]


def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob


def inner_discriminator(x):
    Id_h1 = tf.nn.relu(tf.matmul(x, Id_W1) + Id_b1)
    Id_logit = tf.matmul(Id_h1, Id_W2) + Id_b2
    Id_prob = tf.nn.sigmoid(Id_logit)

    return Id_prob, Id_logit

def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit





In [47]:


G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

Id_prob, Id_logit = inner_discriminator(G_sample)

# D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
# G_loss = -tf.reduce_mean(tf.log(D_fake))

# Alternative losses:
# -------------------
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake

Id_loss = tf.reduce_mean(tf.losses.mean_squared_error(predictions=Id_logit, labels=D_logit_fake))

G_loss_D = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))
G_loss_Id = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Id_logit, labels=tf.ones_like(Id_logit)))


D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
Id_solver = tf.train.AdamOptimizer().minimize(Id_loss, var_list=theta_Id+theta_G)
G_solver_D = tf.train.AdamOptimizer().minimize(G_loss_D, var_list=theta_G+theta_Id)
G_solver_Id = tf.train.AdamOptimizer().minimize(G_loss_Id, var_list=theta_G+theta_Id)




In [48]:
mb_size = 128
Z_dim = 100

if not os.path.exists('MNIST_data'):
    mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

import shutil

out_dir = "out3"
if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
os.makedirs(out_dir)




In [51]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

tom = True
number_of_epochs = 10000
log_steps = 100
i = 0
for it in range(number_of_epochs):
    if it % log_steps == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig(out_dir+'/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    if tom:
        Id_prob_curr = np.zeros(mb_size)
        #while np.mean(Id_prob_curr) < 0.5:
        _, G_loss_Id_curr, Id_prob_curr = sess.run([G_solver_Id, G_loss_Id,Id_prob], feed_dict={Z: sample_Z(mb_size, Z_dim)})
               #print('Id_prob_curr: {:.4}'. format(np.mean(Id_prob_curr)))

    
    #    _, Id_loss_curr = sess.run([Id_solver, Id_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})


    _, _,D_loss_curr = sess.run([D_solver, Id_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _,G_loss_D_curr = sess.run([G_solver_D,G_loss_D], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % log_steps == 0:
        print('Iter: {}'.format(it))
        if tom:
            print('Id loss: {:.4}'. format(Id_loss_curr))
            print('G_loss_Id: {:.4}'.format(G_loss_Id_curr))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss_D: {:.4}'.format(G_loss_D_curr))
        print()

Iter: 0
Id loss: 0.9417
G_loss_Id: 0.2932
D loss: 1.692
G_loss_D: 1.662

Iter: 100
Id loss: 0.9417
G_loss_Id: 0.251
D loss: 0.09914
G_loss_D: 4.22

Iter: 200
Id loss: 0.9417
G_loss_Id: 0.2922
D loss: 0.02367
G_loss_D: 5.373

Iter: 300
Id loss: 0.9417
G_loss_Id: 0.3128
D loss: 0.0369
G_loss_D: 5.709

Iter: 400
Id loss: 0.9417
G_loss_Id: 0.3585
D loss: 0.04791
G_loss_D: 6.508

Iter: 500
Id loss: 0.9417
G_loss_Id: 0.4185
D loss: 0.006079
G_loss_D: 6.79

Iter: 600
Id loss: 0.9417
G_loss_Id: 0.4866
D loss: 0.004794
G_loss_D: 8.54

Iter: 700
Id loss: 0.9417
G_loss_Id: 0.5464
D loss: 0.002962
G_loss_D: 10.6

Iter: 800
Id loss: 0.9417
G_loss_Id: 0.7858
D loss: 0.09063
G_loss_D: 17.53

Iter: 900
Id loss: 0.9417
G_loss_Id: 0.6628
D loss: 0.004552
G_loss_D: 13.39

Iter: 1000
Id loss: 0.9417
G_loss_Id: 0.7143
D loss: 0.03288
G_loss_D: 19.26

Iter: 1100
Id loss: 0.9417
G_loss_Id: 0.4895
D loss: 0.02526
G_loss_D: 11.19

Iter: 1200
Id loss: 0.9417
G_loss_Id: 0.3825
D loss: 0.002742
G_loss_D: 7.703

I